# **Overview of Plotly for Data Visualization and the Dash Framework**

## **Install and Libraries**

In [1]:
%%bash

pip install plotly==5.4 dash jupyter-dash

mkdir assets/
wget https://gist.githubusercontent.com/escape-velocity-labs/17c3ba12aef542afe5055a859e0fbd98/raw/af9e2431fb69c76618aeec9940cd5fd6deab8b1f/style.css -O assets/style.css
wget https://gist.githubusercontent.com/escape-velocity-labs/81b8225d72306541018bf868f3a23436/raw/c14fdac538abc45f8e19c4828bb015aa931c41c5/web_data.csv -O web_data.csv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


--2024-06-15 08:40:04--  https://gist.githubusercontent.com/escape-velocity-labs/17c3ba12aef542afe5055a859e0fbd98/raw/af9e2431fb69c76618aeec9940cd5fd6deab8b1f/style.css
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3793 (3.7K) [text/plain]
Saving to: ‘assets/style.css’

     0K ...                                                   100% 30.1M=0s

2024-06-15 08:40:04 (30.1 MB/s) - ‘assets/style.css’ saved [3793/3793]

--2024-06-15 08:40:04--  https://gist.githubusercontent.com/escape-velocity-labs/81b8225d72306541018bf868f3a23436/raw/c14fdac538abc45f8e19c4828bb015aa931c41c5/web_data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist

In [2]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

## **Dashboard**

In [3]:
app = JupyterDash(__name__)
app.title = 'Web Site Analytics'

/usr/local/lib/python3.10/dist-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [4]:
dataset = pd.read_csv('web_data.csv')
dataset.head()

,date,time,weekday,os,country,page
0,2021-10-01,00:00:00,Friday,Windows,United States,Detail
1,2021-10-01,00:00:00,Friday,Mac,United Kingdom,Shopping Cart
2,2021-10-01,00:00:00,Friday,Linux,Brazil,Detail
3,2021-10-01,00:00:00,Friday,Windows,Canada,Detail
4,2021-10-01,00:00:00,Friday,Linux,Germany,Landing


## **Colors**

In [5]:

colors = ['#3498DB', '#27AE60', '#F39C12', '#E67E22', '#E74C3C', '#9B59B6', '#E91E63', '#ECF0F1']
custom_theme = pio.templates['plotly_dark']

custom_theme.layout.update({
    'paper_bgcolor': '#1f2630',
    'plot_bgcolor': '#1f2630',
    'colorway': colors,
    'font': {
        'color': '#2cfec1'
    },
    'margin': {
        'l': 75,
        'r': 50,
        'b': 100,
        't': 75
    }
})

pio.templates.default = custom_theme


## **Header**

In [6]:
header = html.Div(
    id='header',
    children=[
        html.H4('Web Site Traffic Analytics'),
        html.P(
            id = 'description',
            children = 'Dash learning'
        )
    ]
)

## **First Graph - Evolution of Websites**

In [7]:
visits_per_day = dataset.value_counts('date', sort=False)
visits_per_day.head()

date
2021-10-01    307
2021-10-02    308
2021-10-03    342
2021-10-04    325
2021-10-05    331
Name: count, dtype: int64

In [8]:
trend_title = html.P(className='chart-header', children='Website visits per day')

In [9]:
labels = {'date': 'Date', 'value': 'Value'}
trend_figure = px.line(visits_per_day, labels=labels, markers=True)
trend_figure.layout.update(showlegend=False)
trend_graph = dcc.Graph(figure=trend_figure, className='graph')

##**First Row**

In [10]:
first_row = html.Div(
    className='graph-row',
    children=[
              html.Div(
            className='graph-container',
            children=[
                trend_title,
                trend_graph
            ]
        )
    ]
)


## **Second Graph - Sales Funnel**

In [11]:
funnel_title = html.P(className='chart-header', children='Sales funnel')

In [12]:
@app.callback(
    Output('funnel-graph', 'figure'),
    Input('os-dropdown', 'value'),
    Input('country-dropdown', 'value')
)
def update_funnel_graph(os, country):
    subset = dataset
    if os:
        subset = subset[subset['os'] == os]
    if country:
        subset = subset[subset['country'] == country]

    visits_per_page = subset['page'].value_counts().reset_index()
    visits_per_page.columns = ['page', 'visits']

    fig = px.funnel(visits_per_page, x='visits', y='page', color='page')
    return fig

In [13]:
os_dropdown = dcc.Dropdown(
    id='os-dropdown',
    placeholder='Select an OS',
    options=[
        {'label': 'Windows', 'value': 'Windows'},
        {'label': 'Android', 'value': 'Android'},
        {'label': 'Mac', 'value': 'Mac'},
        {'label': 'IOS', 'value': 'IOS'},
        {'label': 'Linux', 'value': 'Linux'}
    ],
)

country_dropdown = dcc.Dropdown(
    id='country-dropdown',
    placeholder='Select a country',
    options=[
        {'label':'United States', 'value':'United States'},
        {'label':'Canada', 'value':'Canada'},
        {'label':'United Kingdom', 'value':'United Kingdom'},
        {'label':'Australia', 'value':'Australia'},
        {'label':'New Zealand', 'value':'New Zealand'},
        {'label':'Brazil', 'value':'Brazil'},
        {'label':'Russia', 'value':'RUssia'},
        {'label':'Germany', 'value':'Germany'},
        {'label':'Mexico', 'value':'Mexico'},

    ]
)

In [14]:
funnel_controls = html.Div(
    className='controler_row',
    children=[
             os_dropdown,
             country_dropdown
    ]
)

In [15]:
funnel_graph = dcc.Graph(id='funnel-graph', className='graph')

## **Third Graph - Distribution of Visits by Atribute**

In [16]:
pie_title = html.P(className='chart-header', children='Visit distribution')

In [17]:
pie_graph = dcc.Graph(id='pie-graph')

In [18]:
@app.callback(
    Output('pie-graph', 'figure'),
    Input('radio-buttons', 'value')
)
def update_pie_graph(value):
    fig = px.pie(dataset, names=value)
    return fig

In [19]:
radio_buttons = dcc.RadioItems(
    id='radio-buttons',
    options=[
        {'label': 'Operating System', 'value': 'os'},
        {'label': 'Country', 'value': 'country'},
    ],
    value='os',
    labelStyle={'display': 'inline-block'}
)



In [20]:
pie_controls = html.Div(
    className='controler_row',
    children=[
             radio_buttons
    ]
)

## **Second Row**

In [21]:
second_row = html.Div(
    className='graph-row',
    children=[
              html.Div(
            className='graph-container',
            children=[
                funnel_title,
                funnel_controls,
                funnel_graph
            ]
        ),
        html.Div(
            className='graph-container',
            children=[
                pie_title,
                pie_controls,
                pie_graph
            ]
        )
    ]
)

## **Fourth Graph - Visits by Date and Time**

In [22]:
visits_day_time = dataset.groupby(['weekday', 'time']).size().reset_index(name='counts')
visits_day_time.head()

,weekday,time,counts
0,Friday,00:00:00,44
1,Friday,01:00:00,48
2,Friday,02:00:00,53
3,Friday,03:00:00,45
4,Friday,04:00:00,55


In [23]:
visits_day_time =  visits_day_time.pivot( values='counts', index='time', columns='weekday')
visits_day_time.head()

weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
time,,,,,,,
00:00:00,44,38,42,39,47,41,43
01:00:00,48,46,53,48,46,53,45
02:00:00,53,44,58,46,51,44,37
03:00:00,45,35,50,54,35,39,66
04:00:00,55,46,43,51,41,45,46


In [24]:
heatmap_title = html.P(className='chart-header', children='Key activity periods')

In [25]:
heatmap_figure = go.Figure(
    data=go.Heatmap(
         x=visits_day_time.columns.tolist(),
         y=visits_day_time.index.tolist(),
         z=visits_day_time.values.tolist(),
        autocolorscale=True,
        xgap=4,
        ygap=1

   )
)

layout = heatmap_figure.layout.yaxis.update(autorange='reversed', tickvals=[0,6,12,18], ticktext=['Morning', 'Afternoon', 'Evening', 'Night'])



In [26]:
heatmap_graph = dcc.Graph(figure=heatmap_figure, className='graph')

## **Fifth Graph - Distibution by Country**

In [27]:
country_title = html.P(className='chart-header', children='Visits by country')

In [28]:
country_figure = px.histogram(dataset, x='country', color='country')
layout = country_figure.layout.update(showlegend=False)

In [29]:
country_graph = dcc.Graph(figure=country_figure, className='graph')

# **Third Row**

In [30]:
third_row = html.Div(
    className='graph-row',
    children=[
        html.Div(
            className='graph-container',
            children=[
                heatmap_title,
                heatmap_graph
            ]
        ),
        html.Div(
            className='graph-container',
            children=[
                country_title,
                country_graph
            ]
        )
    ]
)


In [31]:
app.layout = html.Div(
    id='root',
    children=[
        header,
        first_row,
        second_row,
        third_row


    ]
)

In [32]:
app.run_server(debug= True, mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>